In [9]:
from QuantLib import *

today = Date(20, 9, 2016)
calendar = TARGET()
conv1 = ModifiedFollowing
conv2 = Unadjusted
frequency = Semiannual
Settings.instance().evaluationDate = today
settlementDate = calendar.advance(today,2,Days,conv1)  # business day/ effective date

terminationDate = [ calendar.advance(settlementDate,
                               n, Years,conv2) 
                for n in range(1, 31) ]
# Unadjusted

print Settings.instance().evaluationDate


September 20th, 2016


In [10]:
tenor = Period(6,Months)
schedule = [ Schedule(settlementDate,terminationDate[n],tenor,calendar,
             conv1,conv1,DateGeneration.Forward, False)
                for n in range(30) ]

In [11]:
ois = {   (1,Days): 0.004,
          (1,Weeks): 0.004,
          (2,Weeks): 0.004,
          (3,Weeks): 0.00417,
          (1,Months): 0.004175,
          (2,Months): 0.00429,
          (3,Months): 0.004366,
          (4,Months): 0.004515,
          (5,Months): 0.00472,
          (6,Months): 0.00485,
          (9,Months): 0.00518,
          (12,Months): 0.005445,
          (18,Months): 0.005801,
          (2,Years): 0.006070,
          (3,Years): 0.006560,
          (4,Years): 0.007010,
          (5,Years): 0.007490,
          (7,Years): 0.012512,
          (10,Years): 0.013925,
          (12,Years): 0.014701,
          (15,Years): 0.015529,
          (20,Years): 0.016385,
          (25,Years): 0.016780,
          (30,Years): 0.016989,
          (40,Years): 0.016995,
          (50,Years): 0.017085}

for n,unit in ois.keys():
    ois[(n,unit)] = SimpleQuote(ois[(n,unit)])

# term structure handles
#RelinkableHandle<YieldTermStructure>
discountTermStructure = RelinkableYieldTermStructureHandle()
forecastTermStructure = RelinkableYieldTermStructureHandle() 

index = USDLibor(tenor,forecastTermStructure) 

floatingLegDayCounter = index.dayCounter()
fixedLegDayCounter = Thirty360()

oisHelpers = [ SwapRateHelper(QuoteHandle(ois[(n,unit)]), # handle<quote>
                               Period(n,unit), calendar,
                               frequency, conv1,
                               fixedLegDayCounter, index)
                for n, unit in ois.keys() ]
print (floatingLegDayCounter)

Actual/360 day counter


In [12]:
# term-structure construction
oisCurve = PiecewiseFlatForward(settlementDate, oisHelpers, Actual360())

discountTermStructure.linkTo(oisCurve)
forecastTermStructure.linkTo(oisCurve)

In [13]:
swaps = { (1,Years): 0.009,
          (2,Years): 0.00975,
          (3,Years): 0.01040,
          (4,Years): 0.01099,
          (5,Years): 0.0116,
          (6,Years): 0.01224,
          (7,Years): 0.01284,
          (8,Years): 0.01343,
          (9,Years): 0.01390,
          (10,Years): 0.01438,
          (15,Years): 0.01610,
          (20,Years): 0.01707,
          (25,Years): 0.01752,
          (30,Years): 0.01778}
for n,unit in swaps.keys():
    swaps[(n,unit)] = SimpleQuote(swaps[(n,unit)])
    

In [14]:
PricingEngine = DiscountingSwapEngine(discountTermStructure)
length = 1
fixedRate = swaps[(length,Years)].value()
spread = 0.0
fixingDays = 2
nominal = 100
payFixed = True

In [15]:
# Swap index
irswap = VanillaSwap(VanillaSwap.Payer, nominal,
                   schedule[length-1], fixedRate, fixedLegDayCounter,
                   schedule[length-1], index, spread,
                   floatingLegDayCounter)

In [16]:
irswap.setPricingEngine(PricingEngine)
print (irswap.NPV())

-0.354105945562
